In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd

In [ ]:
with open('../../../build/tmp.log', 'r') as f:
    data = f.read()
    log_by_key = data.split("Eliminating key ")[1:]
    log_by_key = {block.split('\n')[0]: block for block in log_by_key}
    print(log_by_key.keys())

In [ ]:
tmp = log_by_key['x49']
def from_until(string, start, end):
    return string.split(start)[1].split(end)[0]

def parse_log_section(log_of_key: str):
    joint = log_of_key.split('Passing on joint')[1]
    separator = joint.splitlines()[0][3:]
    C = from_until(joint, 'Cost:C:', 'xc:')
    xc = from_until(joint, 'xc:', '\n')
    inequalities = [line.strip().split(': ')[1] for line in joint.splitlines() if 'Inequality' in line]
    abcdef = np.array([list(map(float, line.split())) for line in C.strip().split("\n")])
    xc = np.array([[float(x)] for x in xc.split()])
    xc = np.array([-999, *xc.flatten(), 999]).reshape(-1, 1)
    def parse_inequality(string):
        coef1 = float(string.split('*')[0])
        rhs = float(string.split('=')[1])
        return coef1, rhs
    # inequalities = [parse_inequality(line) for line in inequalities]
    return separator, xc, abcdef, inequalities

separator, xc, abcdef, inequalities = parse_log_section(tmp)
print(separator, xc.shape, abcdef.shape, inequalities)

In [ ]:
def evaluate_quad(xc, abcdef, num_points=500):
    if xc.shape[0] <= 3:
        xvals = np.linspace(-1, 1, num_points)
    else:
        xvals = np.linspace(xc[1, 0]+1e-9, xc[-2, 0]-1e-9, num_points)
    # xvals = np.linspace(-1, 1, num_points)
    quad = abcdef[:, [0, 3, 5]]
    funcs = [np.poly1d(q) for q in quad]

    conditions = (xvals > xc[:-1]) & (xvals < xc[1:])
    y = np.piecewise(xvals, conditions, funcs)
    return xvals, y

x, y = evaluate_quad(xc, abcdef)

In [ ]:
px.scatter(xc[1:-1]).update_layout(height=200, width=500, margin=dict(l=0,r=0,t=0,b=0)).show()

In [ ]:
# Create a plot with a line
def plot(x, y, xc):
    fig = px.scatter(x=x, y=y)
    fig.update_traces(mode="lines")
    # Draw a vertical line for each xc
    for x in xc[1:-1].flatten():
        fig.add_vline(x=x, line_dash="dash")
    # fig.add_scatter(x=xc[1:-1].flatten(), y=np.zeros_like(xc[1:-1]), mode="markers")
    return fig
plot(x, y, xc)

In [ ]:
results = {}
sep = 'x49'
while True:
    print(sep, end=' ')
    try:
        sep, xc, abcdef, ineq = parse_log_section(log_by_key[sep])
        x, y = evaluate_quad(xc, abcdef, num_points=100)
        results[sep] = (x, y, xc, abcdef, ineq)
    except:
        print()
        break
print(f'Finished on {sep}')


In [ ]:
# results[sep] = (x, y, xc, abcdef, ineq)
# Create a dataframe with the results, with columns "key", "x", "y", and flattening out x, y since they are both Nx1 arrays
df = pd.DataFrame([(key, *results[key][:2]) for key in results.keys()], columns=['key', 'x', 'y']).apply(pd.Series.explode)
display(df[::1000].head())
xc_df = pd.DataFrame([(int(key[1:]), results[key][2][1:-1].flatten()) for key in results.keys()], columns=['key', 'xc']).explode('xc')
display(xc_df.head())

In [ ]:
display(results['x4'][2].tolist())
display(results['x4'][3].tolist())

In [ ]:
# Create a stacked plotly scatter plot for the following x values
# Import make_suplots
from plotly.subplots import make_subplots


fig = px.scatter(df, x='x', y='y', facet_col='key', facet_col_wrap=5, height=1000, facet_col_spacing=0.01, facet_row_spacing=0.01)
fig.update_traces(mode="lines")
fig.update_layout(margin=dict(l=0,r=0,t=0,b=0))
fig.update_yaxes(range=[-0.05, 0.05])
for axis in fig.layout:
    if axis.startswith("yaxis"):
        fig.layout[axis]['matches'] = None
# Draw a vertical line for each xc
# for index, row_ in xc_df[:100].iterrows():
#     i = row_['key']
#     col = i % 5 + 1  # Assuming facet_col_wrap=5
#     row = 9 - i // 5 + 1
#     fig.add_vline(x=row_['xc'], line_dash="dash", col=col, row=row)
fig.show()

# for i in [5, 15, 25, 35, 45]:
#     x, y, xc, abcdef, ineq = results[f'x{i}']
#     fig = plot(x, y, xc)
#     fig.update_layout(title=f'x{i}', height=200, width=400, margin=dict(l=0,r=0,t=30,b=0))
#     fig.show()

In [ ]:
x, y, xc, abcdef, _ = results['x37']
with np.printoptions(suppress=True, linewidth=300):
    display(np.hstack((xc[1:], abcdef)))